In [10]:
import pandas as pd
import os
import numpy
from collections import defaultdict
import rpy2.robjects as robjects

In [11]:
robjects.r('''
library(ROCR)
getObs_ch2 <- function(ls,threshold = F) {
  combi <- unique(ls$COMBINATION_ID)
  cell <- unique(ls$CELL_LINE)
  mat <- matrix(NA, nrow=length(combi), ncol=length(cell),
                dimnames=list(combi, cell))
  for (i in 1:nrow(ls)) 
    mat[as.character(ls$COMBINATION_ID[i]), 
        as.character(ls$CELL_LINE[i])] <- ls$SYNERGY_SCORE[i]
  
  if (is.numeric(threshold)) {
    mat[mat <= threshold] = 0
    mat[mat > threshold ] = 1
  }
  return(mat)
}

getPred_ch2 <- function(pred) {
  if (all(row.names(pred) == c(1:nrow(pred)))) {
    row.names(pred) = pred[,1]
    pred = pred[,-1]
  }
  pred <- as.matrix(pred) 
  return(pred)
}

getNegLog10pVal_ch2 <- function(fit, obs) {
  s <- 0
  if (!is.na(fit$coefficients[2]) & sum(!is.na(obs)) > 2)
      s <- -log10(anova(fit)['pred','Pr(>F)'])
  return(s)
}

getPrecision_ch2 <- function(pred,obs, threshold=30) {
  obs <- read.csv(obs)
  obs <- getObs_ch2(obs,threshold)
  
  pred <- read.csv(pred,stringsAsFactors=F,check.names = F)
  pred <- getPred_ch2(pred)  
  pred <- pred[match(row.names(obs),row.names(pred)),]
  pred <- pred[,match(colnames(obs),colnames(pred))]

  #Remove all NA's
  pred <- as.numeric(pred)[!is.na(obs)]
  obs <- as.numeric(obs)[!is.na(obs)]

  preds <- prediction(pred,obs)
  prec <- performance(preds,"prec") #precision (Acc + )
  sens <- performance(preds,"sens") #True positive rate (Sensitivity) (Cov +)
  npv <- performance(preds,"npv") #Negative predictive value (Acc - )
  spec <- performance(preds,"spec") #True negative rate(specificity) (Cov -)
  auc <- performance(preds,"auc") #Area under curve (AUC)
  phi <- performance(preds,"phi") #phi correlation coefficient, (matthews)
  aupr <- performance(preds, "prec", "rec") #Area under precision recall (AUPR)
  
  prec_val <- unlist(prec@y.values)[2]
  sens_val <- unlist(sens@y.values)[2]
  npv_val <- unlist(npv@y.values)[2]
  spec_val <- unlist(spec@y.values)[2]
  auc_val <- unlist(auc@y.values)
  phi_val <- unlist(phi@y.values)[2]
  BAC <- (sens_val + spec_val)/2
  F1 <- 2*preds@tp[[1]][2]/(2*preds@tp[[1]][2] + preds@fn[[1]][2] + preds@fp[[1]][2])
  aupr_val <- unlist(aupr@y.values)[2]

  return(round(c(prec=prec_val,
                 sens = sens_val,
                 npv = npv_val,
                 spec=spec_val,
                 auc=auc_val,
                 phi=phi_val,
                 BAC=BAC,
                 F1=F1,                 
                 aupr=aupr_val),2))

  
}

getOneDimScore_ch2 <- function(pred,obs, confidence="none", topX=10, rows=T) {
  obs <- read.csv(obs)
  obs <- getObs_ch2(obs)
  
  pred <- read.csv(pred,stringsAsFactors=F,check.names = F)
  pred <- getPred_ch2(pred)
  pred <- pred[match(row.names(obs),row.names(pred)),]
  pred <- pred[,match(colnames(obs),colnames(pred))]
  
  n <- ncol(obs)
  if (rows)
    n <- nrow(obs)
  
  s <- c()
  for (i in 1:n) {
    if (rows) {
      fit <- aov(obs[i,] ~ pred[i,])
      nlp <- getNegLog10pVal_ch2(fit,obs[i,])
    } else {
      fit <- aov(obs[,i] ~ pred[,i]) 
      nlp <- getNegLog10pVal_ch2(fit,obs[,i])
    }
    
    sign <- 1
    if (mean(obs[pred==1], na.rm=T) < mean(obs[pred==0], na.rm=T))
      sign <- -1
    
    s <- c(s, sign * nlp)
  }
  
  if (!file.exists(confidence))
    return(round(c(mean=mean(s),
             ste=sd(s)),2))
  
  confidence <- read.csv(confidence,stringsAsFactors=F,check.names = F)
  confidence <- getPred_ch2(confidence)
  confidence <- confidence[match(row.names(obs),row.names(confidence)),]
  confidence <- confidence[,match(colnames(obs),colnames(confidence))]
  
  if (rows) {
    idx <- order(rowSums(confidence), decreasing = T)[1:round(topX * (nrow(confidence) / 100))]
  } else {
    idx <- order(colSums(confidence), decreasing = T)[1:round(topX * (ncol(confidence) / 100))]
  }
  
  return(round(c(mean=mean(s[idx]),
           ste=sd(s[idx])),2))
}

getGlobalScore_ch2 <- function(pred,obs) { 
  obs <- read.csv(obs)
  obs <- getObs_ch2(obs)
  
  pred <- read.csv(pred,stringsAsFactors=F,check.names = F)
  pred <- getPred_ch2(pred)
  pred <- pred[match(row.names(obs),row.names(pred)),]
  pred <- pred[,match(colnames(obs),colnames(pred))]
  
  # regress out combination bias
  cov <- rep(rownames(obs), ncol(obs))
  
  c0 <- rep(rownames(obs), ncol(obs))
  c1 <- as.vector(matrix(colnames(obs), ncol=ncol(obs), nrow=nrow(obs), byrow=T))
  
  obs <- as.vector(obs)
  pred <- as.vector(pred)
  
  # run anove with combination label as covariate
  fit <- aov(obs ~ c0 + c1 + pred)
  pVal <- -log10(anova(fit)['pred','Pr(>F)'])
  
  sign <- 1
  if (mean(obs[pred==1], na.rm=T) < mean(obs[pred==0], na.rm=T))
    sign <- -1
  
  return(round(sign * pVal,2))
}
''')

<SignatureTranslatedFunction - Python:0x000000000A63ACC8 / R:0x000000000DB77F00>

In [12]:
cellLineBool=1
drugCombBool=0
maxconcBool=1
einfBool=1
drugTargetBool=1
mutBool=1

outputFileName=''
if cellLineBool: outputFileName+='cellID_'
if drugCombBool==1: outputFileName+='drugCombID_'
elif drugCombBool==0: outputFileName+='drugID_'
if maxconcBool: outputFileName+='maxconc_'
if einfBool: outputFileName+='einf_'
if drugTargetBool: outputFileName+='drugTarget_'
if mutBool: outputFileName+='mut_'
outputFileName+='sub2_result'
print outputFileName

libfmId=0
groupData=[]
groupId=0

if cellLineBool:
    cl={}
    pr=pd.read_csv('input/ch1_train_combination_and_monoTherapy.csv')
    for i,name in enumerate(set(pr['CELL_LINE'])):
        cl[name]=i
        groupData.append(str(groupId))

    libfmId+=len(cl)
    groupId+=1
    print 'cell line 개수:',len(cl)
    
if drugCombBool==1:
    dc={}
    pr=pd.read_csv('input/ch1_train_combination_and_monoTherapy.csv')
    for i,name in enumerate(set(pr['COMBINATION_ID']),start=libfmId):
        dc[name]=i
        groupData.append(str(groupId))
    libfmId+=len(dc)
    groupId+=1
    print 'drug combination 개수: ',len(dc)
elif drugCombBool==0:
    drugName={}
    pr=pd.read_csv('drug/Drug_info_release.csv')
    for i,name in enumerate(set(pr['ChallengeName']),start=libfmId):
        drugName[name]=i
        groupData.append(str(groupId))
    libfmId+=len(drugName)
    groupId+=1
    print 'drug 개수:',len(drugName)
else:
    print 'no drug'

if maxconcBool:
    maxconca=libfmId
    maxconcb=libfmId+1
    libfmId+=2
    groupData.extend([str(groupId),str(groupId)])
    groupId+=1
    print 'max conc a,b'

if einfBool:
    einfaId=libfmId
    einfbId=libfmId+1
    libfmId+=2
    groupData.extend([str(groupId),str(groupId)])
    groupId+=1
    print 'einf a,b'
    
if drugTargetBool:
    pr=pd.read_csv('drug/woDNA.csv')
    dtl=set() # drug target list
    for targets in pr['Target']:
        if not pd.isnull(targets):
            targetSplit=targets.split(',')
            for i in range(len(targetSplit)):
                if '*' not in targetSplit[i]:
                    dtl.add(targetSplit[i].strip())

    dtId={}
    for i,d0 in enumerate(dtl,start=libfmId):
        dtId[d0]=i
        groupData.append(str(groupId))
    
    ddt={}
    for row in pr.itertuples():
        tmp=set()
        if not pd.isnull(row[2]):
            targetSplit=row[2].split(',')
            for i in range(len(targetSplit)):
                if '*' not in targetSplit[i]:
                    tmp.add(dtId[targetSplit[i].strip()])
        ddt[row[1]]=tmp
    
    print 'drug target 개수: ',len(dtl)
    libfmId+=len(dtl)
    groupId+=1

if mutBool:
    pr=pd.read_csv('drug/woDNA.csv')
    drugTargetSetTmp=set() # drug target list
    for targets in pr['Target']:
        if not pd.isnull(targets):
            targetSplit=targets.split(',')
            for i in range(len(targetSplit)):
                if '*' not in targetSplit[i]:
                    drugTargetSetTmp.add(targetSplit[i].strip())
    
    dtId2={}
    for i,d0 in enumerate(drugTargetSetTmp,start=libfmId):
        dtId2[d0]=i
        groupData.append(str(groupId))    
    
    libfmId+=len(dtId2)
    
    clMut={}
    pr=pd.read_csv('mutation/mutations.csv')
    for row in pr[(pr['Tumour.origin']=='primary')].itertuples():
        if row[1] in dtId2: 
            if row[1]=='BRAF' and row[15]=='p.V600E':
                if row[5] in clMut: clMut[row[5]].add(dtId2['BRAF_V600E'])
                else: clMut[row[5]]=set([dtId2['BRAF_V600E']])
            else:
                if row[5] in clMut: 
                    clMut[row[5]].add(dtId2[row[1]])
                else: clMut[row[5]]=set([dtId2[row[1]]])

    clNotInMut=defaultdict(set)
    for cellLineVar in clMut:
        for drugTargetIdVar in dtId2.values():
            if drugTargetIdVar not in clMut[cellLineVar]:
                clNotInMut[cellLineVar].add(drugTargetIdVar)
        
    print 'mutations 개수: ',len(dtId2)
    groupId+=1

print '총 feature 개수: ',libfmId

cellID_drugID_maxconc_einf_drugTarget_mut_sub2_result
cell line 개수: 85
drug 개수: 119
max conc a,b
einf a,b
drug target 개수:  96
mutations 개수:  96
총 feature 개수:  400


In [ ]:
for inputIndex in range(3):
    trainDir='input/sub2_train_set(QA=1 limit)_'+str(inputIndex)+'.csv'
    testDir='input/sub2_test_set(QA=1 limit)_'+str(inputIndex)+'.csv'
    print trainDir,testDir
    libfmTrain='libfmInput/sub2Train.libfm'
    libfmTest='libfmInput/sub2Test.libfm'
    metaDir='libfmInput/sub2meta.txt'

    with open(metaDir,'w') as fw:
        for item in groupData:
            fw.write(item+'\n')

    pr=pd.read_csv(trainDir)
    featureString=''
    with open(libfmTrain,'w') as fw:
        for row in pr.itertuples():
            featureString+=str(row[12])+' ' # score

            if cellLineBool: featureString+=str(cl[row[1]])+':1 ' # cell line

            if mutBool:
                if row[1] in clMut:
                    for x in clMut[row[1]]: featureString+=str(x)+':1 '
                if row[1] in clNotInMut:
                    for x in clNotInMut[row[1]]: featureString+=str(x)+':-1 '

            if drugCombBool==1:featureString+=str(dc[row[14]])+':1 '
            elif drugCombBool==0: featureString+=str(drugName[row[2]])+':1 '+str(drugName[row[3]])+':1 '

            if drugTargetBool:
                u=ddt[row[2]].intersection(ddt[row[3]])
                for x in u: featureString+=str(x)+':2 '
                for x in ddt[row[2]]-u: featureString+=str(x)+':1 '
                for x in ddt[row[3]]-u: featureString+=str(x)+':1 '

            if maxconcBool:
                featureString+=str(maxconca)+':'+str(row[4]/pr['MAX_CONC_A'].max())+' '
                featureString+=str(maxconcb)+':'+str(row[5]/pr['MAX_CONC_B'].max())+' '

            if einfBool:
                featureString+=str(einfaId)+':'+str(row[8]/pr['Einf_A'].max())+' '
                featureString+=str(einfbId)+':'+str(row[11]/pr['Einf_B'].max())+' '

            featureString=featureString.strip()
            featureString+='\n'
        fw.write(featureString)

    pr=pd.read_csv('input/ch1_train_combination_and_monoTherapy.csv')
    cell=list(set(pr['CELL_LINE']))
    prTest=pd.read_csv(testDir)
    combi=list(set(prTest['COMBINATION_ID']))

    cellRep=[]
    for x in cell:
        for i in range(len(combi)):cellRep.append(x)

    combiRep=combi*len(cell)

    df=pd.DataFrame({'CELL_LINE':cellRep,'COMBINATION_ID':combiRep})

    for row in prTest[['CELL_LINE','MAX_CONC_A','MAX_CONC_B','Einf_A','Einf_B','COMBINATION_ID']].itertuples():
        df.loc[(df['CELL_LINE']==row[1]) & (df['COMBINATION_ID']==row[6]),'MAX_CONC_A']=row[2]
        df.loc[(df['CELL_LINE']==row[1]) & (df['COMBINATION_ID']==row[6]),'MAX_CONC_B']=row[3]
        df.loc[(df['CELL_LINE']==row[1]) & (df['COMBINATION_ID']==row[6]),'Einf_A']=row[4]
        df.loc[(df['CELL_LINE']==row[1]) & (df['COMBINATION_ID']==row[6]),'Einf_B']=row[5]

    featureString=''
    with open(libfmTest,'w') as fw:
        for row in df.itertuples():
            featureString+='0 ' # synergy scroe
            if cellLineBool: featureString+=str(cl[row[1]])+':1 ' # cell line
            if mutBool:
                if row[1] in clMut:
                    for x in clMut[row[1]]: featureString+=str(x)+':1 '
                if row[1] in clNotInMut:
                    for x in clNotInMut[row[1]]: featureString+=str(x)+':-1 '

            drug=row[2].split('.') # drug[0] = compound a, drug[1] = compound b
            if drugCombBool==1:
                if row[2] in dc:featureString+=str(dc[row[2]])+':1 '
            elif drugCombBool==0: featureString+=str(drugName[drug[0]])+':1 '+str(drugName[drug[1]])+':1 '

            if drugTargetBool:
                u=ddt[drug[0]].intersection(ddt[drug[1]])
                for x in u: featureString+=str(x)+':2 '
                for x in ddt[drug[0]]-u: featureString+=str(x)+':1 '
                for x in ddt[drug[1]]-u: featureString+=str(x)+':1 '

            if maxconcBool:
                if not pd.isnull(row[3]):
                    featureString+=str(maxconca)+':'+str(row[3]/pr['MAX_CONC_A'].max())+' '
                    featureString+=str(maxconcb)+':'+str(row[4]/pr['MAX_CONC_B'].max())+' '

            if einfBool:
                if not pd.isnull(row[5]):
                    featureString+=str(einfaId)+':'+str(row[5]/pr['Einf_A'].max())+' '
                    featureString+=str(einfbId)+':'+str(row[6]/pr['Einf_B'].max())+' '

            featureString=featureString.strip()
            featureString+='\n'
        fw.write(featureString)


    def runLibfmTest(i=0,dim='10',iter1='100',stdev='0.001'):
        os.system("libfm -task r -train "+libfmTrain+" -test "+libfmTest+" -dim '1,1,"+dim+"' -iter "+iter1+" -meta "+metaDir
                  +" -init_stdev "+stdev+" -out prediction\\"+outputFileName+str(i))

    pred=df[['CELL_LINE','COMBINATION_ID']]

    for dim in range(20,21):
        for iter1 in range(250,300,10):
            print dim,iter1
            for i in range(10):
                runLibfmTest(i=i,dim=str(dim),iter1=str(iter1))
            num=10
            a=[]
            for i in range(num):
                tmp=[]
                with open('prediction/'+outputFileName+str(i),'r') as fr:
                    for row in fr.readlines():
                        tmp.append(float(row.strip()))
                if len(a)!=0:
                    a=[sum(x) for x in zip(a, tmp)]
                else:
                    a=tmp
            a=map(lambda x: x/num, a)
            a2=[]
            for x in a:
                if x>30:a2.append(1)
                else:a2.append(0)
            pred['PREDICTION']=numpy.asarray(a2)

            pivoted = pred.pivot_table(index='COMBINATION_ID', columns='CELL_LINE', values='PREDICTION')
            pivoted.to_csv('finalSubmissionFile'+str(dim)+'_'+str(iter1)+'_'+'.csv')

            print robjects.r['getGlobalScore_ch2']('finalSubmissionFile'+str(dim)+'_'+str(iter1)+'_'+'.csv',testDir)
            print robjects.r['getOneDimScore_ch2']('finalSubmissionFile'+str(dim)+'_'+str(iter1)+'_'+'.csv',testDir)
            print robjects.r['getPrecision_ch2']('finalSubmissionFile'+str(dim)+'_'+str(iter1)+'_'+'.csv',testDir)

input/sub2_train_set(QA=1 limit)_0.csv input/sub2_test_set(QA=1 limit)_0.csv
20